## Step 1. Import necessary libraries

In [ ]:
# import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import time

import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, multilabel_confusion_matrix
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, GridSearchCV, GroupKFold, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, Normalizer, OneHotEncoder, RobustScaler, StandardScaler
from sklearn.svm import SVC

from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

## Step 2a. Declare user variables

In [ ]:
is_test = False
is_dataset_one_file = False

user_specified_layout = 'layout1'
user_specified_layout_int = int(user_specified_layout[len(user_specified_layout) - 1])
user_specified_clf = 'all'

user_test_size = 0.2
user_random_state = 1

if user_specified_layout_int == 1:
    ith_inp_col = 16
else:
    ith_inp_col = 20

list_filenames = {
    'layout1':'dataset1.csv',
    'layout2':'dataset2.csv',
}
list_columns = {
    'layout1_all':['timestamp','posture_id','posture_label','s01','s02','s03','s04','s05','s06','s07','s08','s09','s10','s11','s12','s13','s14','s15','s16','birth_year','sex','height','weight','bmi','bmi_label','full_name','nth','round'],
    'layout2_all':['timestamp','posture_id','posture_label','s01','s02','s03','s04','s05','s06','s07','s08','s09','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','birth_year','sex','height','weight','bmi','bmi_label','full_name','nth','round'],
    'layout1_cat_inp':[],
    'layout1_num_inp':['s01','s02','s03','s04','s05','s06','s07','s08','s09','s10','s11','s12','s13','s14','s15','s16', 'height','weight','bmi'],
    'layout2_cat_inp':[],
    'layout2_num_inp':['s01','s02','s03','s04','s05','s06','s07','s08','s09','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20', 'height','weight','bmi'],
}
list_positions = ['Yearner_Right', 'Yearner_Left', 'Fetal_Right', 'Fetal_Left', 'Log_Right', 'Log_Left', 'Supine', 'Prone',
                  # 'Empty'
]
list_grouped_positions = [
    'Rights',
    'Lefts',
#     'Fetal_Right',
#     'Fetal_Left',
    'Supine',
    'Prone',
]

# {
#     'Yearner_Right' : 1,
#     'Yearner_Left' : 2,
#     'Fetal_Right' : 3,
#     'Fetal_Left' : 4,
#     'Log_Right' : 5,
#     'Log_Left' : 6,
#     'Supine' : 7,
#     'Prone' : 8,
# }

filename = list_filenames[user_specified_layout]
cols_all = list_columns[user_specified_layout + '_all']
cols_cat_inp = list_columns[user_specified_layout + '_cat_inp']
cols_num_inp = list_columns[user_specified_layout + '_num_inp']
cols_inp = cols_cat_inp + cols_num_inp
cols_drp = list(set(cols_all) - set(cols_inp))

cols_num_inp_std = ['height','weight','bmi']
cols_num_inp_nrm = list(set(cols_num_inp) - set(cols_num_inp_std))

col_grp = 'full_name'
col_trg = 'posture_id'

pd.set_option('display.float_format', lambda x: '%.2f' % x)
sns.reset_orig()

## Step 2b. Declare and prepare needed variables

In [ ]:
if is_dataset_one_file:
    df = pd.read_csv(filename, usecols = cols_all)
else:
    import os
    import glob

    # os.chdir("C:/Users/Julianne/Documents/Notebooks/thesis/data/set_{}".format(user_specified_layout_int))
    os.chdir("C:/Users/Julianne/Downloads/set_{} clean".format(user_specified_layout_int))

    extension = 'csv'
    list_raw_filenames = [i for i in glob.glob('*.{}'.format(extension))]

    df = pd.concat([pd.read_csv(f, usecols = cols_all) for f in list_raw_filenames])

    df = df[df.nth <= 5]
    df = df[df.nth == 3]

    ## export to csv
    # combined_csv.to_csv("dataset{}.csv".format(user_specified_layout_int), index=False, encoding='utf-8-sig')
    
    os.chdir("C:/Users/Julianne/Documents/Notebooks/thesis".format(user_specified_layout_int))

In [ ]:
if is_test:
    df

In [ ]:
if is_test:
    df.info()

In [ ]:
def categorize(row):
    if row['posture_label'] == 'Yearner_Right' or row['posture_label'] == 'Log_Right' or row['posture_label'] == 'Fetal_Right':
        return 100
    elif row['posture_label'] == 'Yearner_Left' or row['posture_label'] == 'Log_Left' or row['posture_label'] == 'Fetal_Left':
        return 200
    else:
        return row['posture_id']

df_inp = df.drop(columns=cols_drp)

X = df_inp

y = df[col_trg].values
y_intermediate = df.apply(lambda row: categorize(row), axis=1).values
groups = df[col_grp].values

# TODO: delete cols_identifying_idx and other calls to this variable
cols_identifying_idx = df_inp.columns.get_indexer(['height','weight','bmi'])

In [ ]:
cols_cat_inp_idx = df_inp.columns.get_indexer(cols_cat_inp)
cols_num_inp_idx = df_inp.columns.get_indexer(cols_num_inp)

In [ ]:
# #apply SelectKBest class to extract top 10 best features
# bestfeatures = SelectKBest(score_func=chi2, k=10)
# fit = bestfeatures.fit(X,y)
# dfscores = pd.DataFrame(fit.scores_)
# dfcolumns = pd.DataFrame(X.columns)
# #concat two dataframes for better visualization 
# featureScores = pd.concat([dfcolumns,dfscores],axis=1)
# featureScores.columns = ['Specs','Score']  #naming the dataframe columns
# print(featureScores.nlargest(10,'Score'))  #print 10 best features

## Step 3. Visualize the dataset

In [ ]:
if is_test:
    df_inp.describe()

In [ ]:
# pd.plotting.scatter_matrix(X, figsize=(10, 10));

In [ ]:
if is_test:
    # Create correlation matrix
    corr_matrix = df.drop(columns=cols_drp).corr()

    # Define mask used to cover squares above diagonal
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

    plt.figure(figsize=(20, 10), facecolor='w', edgecolor='k')
    plt.title('Correlation Matrix - ' + user_specified_layout)
    sns.set(font_scale=1.2)
    sns.heatmap(corr_matrix, cmap='coolwarm', center = 0, annot=True, fmt='.1g', mask=mask)

In [ ]:
steps_cat_inp = [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
]
pipe_cat_inp = Pipeline(steps_cat_inp)

steps_num_inp = [
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
]
pipe_num_inp = Pipeline(steps_num_inp)

ct = ColumnTransformer(transformers=[
          ('categorical', pipe_cat_inp, cols_cat_inp_idx),
          ('numerical', pipe_num_inp, cols_num_inp_idx)
])

In [ ]:
df[col_grp].nunique()

## Step 4. Declare needed functions

In [ ]:
def print_metrics(actual_targets, predicted_targets, target_classes):
    print(accuracy_score(actual_targets, predicted_targets))
#     print(precision_score(actual_targets, predicted_targets, average='micro'))
#     print(recall_score(actual_targets, predicted_targets, average='micro'))
#     print(f1_score(actual_targets, predicted_targets, average='micro'))
#     try:
#         print(classification_report(actual_targets, predicted_targets, target_names=target_classes))
#     except:
#         print(classification_report(actual_targets, predicted_targets))

In [ ]:
def generate_confusion_matrix(cnf_matrix, actual_targets, predicted_targets, target_classes, normalize=False, title='Confusion Matrix'):
    if normalize:
        cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
        print("Confusion Matrix, With Normalized Counts")
        ConfusionMatrixDisplay.from_predictions(actual_targets, predicted_targets,
                                                xticks_rotation = 45, 
                                                normalize='true', 
                                                values_format = '.2f')
    else:
        print("Confusion Matrix, Without Normalized Counts")
        ConfusionMatrixDisplay.from_predictions(actual_targets, predicted_targets,
                                                xticks_rotation = 45)

    plt.title(title)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    return cnf_matrix

In [ ]:
def plot_confusion_matrix(actual_targets, predicted_targets, classifier, target_classes):
    cm = confusion_matrix(actual_targets, predicted_targets)
    np.set_printoptions(precision=2)

    print_metrics(actual_targets, predicted_targets, target_classes)

    # Plot non-normalized confusion matrix
    plt.figure()
    generate_confusion_matrix(cm, actual_targets, predicted_targets, target_classes=target_classes, title='%s Confusion Matrix - Layout %x (Without Normalized Counts)' % (classifier, user_specified_layout_int))
    plt.savefig('CM-L%x-%s-all-nonnormalized.svg' % (user_specified_layout_int, classifier), format='svg', bbox_inches="tight")
    plt.show()

    # Plot normalized confusion matrix
    plt.figure()
    generate_confusion_matrix(cm, actual_targets, predicted_targets, target_classes=target_classes, normalize=True, title='%s Confusion Matrix - Layout %x (With Normalized Counts)' % (classifier, user_specified_layout_int))
    plt.savefig('CM-L%x-%s-all-normalized.svg' % (user_specified_layout_int, classifier), format='svg', bbox_inches="tight")
    plt.show()

In [ ]:
def model(train_x, train_y, test_x, test_y, classifier, target_classes):
    if classifier == 'MLP':
        pipe = Pipeline([
            ('pre', ct), 
#                 ('feature_selection', SelectKBest(score_func=f_regression, k=4)), 
            ('clf', MLPClassifier(max_iter=5000))
        ])
    elif classifier == 'SVC':
        pipe = Pipeline([
            ('pre', ct), 
#                 ('feature_selection', SelectKBest(score_func=f_regression, k=4)), 
            ('clf', SVC(kernel='rbf', random_state=user_random_state))
        ])

    pipe.fit(train_x, train_y)
    predicted_labels = pipe.predict(test_x)

    return predicted_labels

In [ ]:
def evaluate_subgroup(train_x, train_y, test_x, test_y, intermediate_labels, list_ids, subgroup_id, classifier, target_classes):
    train_x_subgroup = train_x[np.isin(train_y, list_ids)]
    train_y_subgroup = train_y[np.isin(train_y, list_ids)]
    test_x_predicted_subgroup = test_x[np.isin(intermediate_labels, subgroup_id)]
    test_y_corresponding_predicted_subgroup = test_y[np.isin(intermediate_labels, subgroup_id)]

    predicted_labels_from_subgroup = []
    if test_x_predicted_subgroup.size:
        predicted_labels_from_subgroup = model(train_x_subgroup, train_y_subgroup, 
                                               test_x_predicted_subgroup, test_y_corresponding_predicted_subgroup, 
                                               classifier, target_classes)
    else:
        print("None of the participant's data was classified as subgroup {}".format(subgroup_id))

    return predicted_labels_from_subgroup, test_y_corresponding_predicted_subgroup

In [ ]:
def evaluate_model(data_x, data_y, data_y_intermediate, classifier, target_classes):
    group_k_fold = GroupKFold(n_splits=df[col_grp].nunique())

    sample = np.array([])
    predicted_targets = np.array([])
    intermediate_actual_targets = np.array([])
    actual_targets = np.array([])
    predicted_targets_rights = np.array([])
    actual_targets_rights = np.array([])
    predicted_targets_lefts = np.array([])
    actual_targets_lefts = np.array([])

    for i, (train_index, test_index) in enumerate(group_k_fold.split(data_x, data_y, groups=groups)):
        train_x, test_x = data_x[train_index], data_x[test_index]
        train_y, test_y = data_y[train_index], data_y[test_index]
        train_y_intermediate, test_y_intermediate = data_y_intermediate[train_index], data_y_intermediate[test_index]
        print('fold %x (height: %.2f, weight: %.2f, bmi: %.2f)' % (i, test_x[0, cols_identifying_idx[0]], test_x[0, cols_identifying_idx[1]], test_x[0, cols_identifying_idx[2]]))

        intermediate_output = model(train_x, train_y_intermediate, test_x, test_y_intermediate, classifier, target_classes)

        predicted_targets = np.append(predicted_targets, intermediate_output)
        intermediate_actual_targets = np.append(intermediate_actual_targets, test_y_intermediate)
        actual_targets = np.append(actual_targets, test_y)

#         cmd = ConfusionMatrixDisplay.from_predictions(test_y_intermediate, predicted_labels, 
#                                                       display_labels = target_classes, 
#                                                       xticks_rotation = 45, 
#                                                       normalize='true', 
#                                                       values_format = '.2f')
#         plt.title('%s Confusion Matrix - Layout %x (height: %.2f, weight: %.2f, bmi: %.2f)' % (classifier, user_specified_layout_int, test_x[0, cols_identifying_idx[0]], test_x[0, cols_identifying_idx[1]], test_x[0, cols_identifying_idx[2]]))
#         plt.tight_layout()
#         plt.savefig('kfold confusion matrices/CM-L%x-%s-height-%.2f, weight-%.2f, bmi-%.2f.svg' % (user_specified_layout_int, classifier, test_x[0, cols_identifying_idx[0]], test_x[0, cols_identifying_idx[1]], test_x[0, cols_identifying_idx[2]]), format='svg', bbox_inches="tight")
#         plt.show()

        output, corresponding_true_y = evaluate_subgroup(train_x, train_y, test_x, test_y, intermediate_output, [1,3,5], 100, classifier, target_classes)

        predicted_targets_rights = np.append(predicted_targets_rights, output)
        actual_targets_rights = np.append(actual_targets_rights, corresponding_true_y)

        output, corresponding_true_y = evaluate_subgroup(train_x, train_y, test_x, test_y, intermediate_output, [2,4,6], 200, classifier, target_classes)

        predicted_targets_lefts = np.append(predicted_targets_lefts, output)
        actual_targets_lefts = np.append(actual_targets_lefts, corresponding_true_y)

#         cmd = ConfusionMatrixDisplay.from_predictions(test_y_corresponding_predicted_rights, predicted_labels_2, 
#                                                       display_labels = ['Yearner Right', 'Log Right', 'Lost'], 
#                                                       xticks_rotation = 45, 
#                                                       normalize='true', 
#                                                       values_format = '.2f')
#         plt.title('%s Confusion Matrix - Layout %x (height: %.2f, weight: %.2f, bmi: %.2f)' % (classifier, user_specified_layout_int, test_x[0, cols_identifying_idx[0]], test_x[0, cols_identifying_idx[1]], test_x[0, cols_identifying_idx[2]]))
#         plt.tight_layout()
# #         plt.savefig('kfold confusion matrices/CM-L%x-%s-height-%.2f, weight-%.2f, bmi-%.2f.svg' % (user_specified_layout_int, classifier, test_x[0, cols_identifying_idx[0]], test_x[0, cols_identifying_idx[1]], test_x[0, cols_identifying_idx[2]]), format='svg', bbox_inches="tight")
#         plt.show()

    plot_confusion_matrix(intermediate_actual_targets, predicted_targets, classifier, target_classes)
    plot_confusion_matrix(actual_targets_rights, predicted_targets_rights, classifier, list_positions)
    plot_confusion_matrix(actual_targets_lefts, predicted_targets_lefts, classifier, list_positions)

    new_predicted_targets = []
    ctr_rights = 0
    ctr_lefts = 0
    for target in predicted_targets:
        if target == 100:
            new_predicted_targets.append(predicted_targets_rights[ctr_rights])
            ctr_rights += 1
        elif target == 200:
            new_predicted_targets.append(predicted_targets_lefts[ctr_lefts])
            ctr_lefts += 1
        else:
            new_predicted_targets.append(target)
            
#     np.putmask(predicted_targets, predicted_targets == 100, predicted_targets_rights)
#     np.putmask(predicted_targets, predicted_targets == 200, predicted_targets_lefts)
#     np.putmask(actual_targets, actual_targets == 100, actual_targets_rights)
#     np.putmask(actual_targets, actual_targets == 200, actual_targets_lefts)

    return actual_targets, new_predicted_targets

## Step 5. Execute

In [ ]:
X = df_inp.to_numpy()

data = X
target = y

In [ ]:
actual_targets, predicted_targets = evaluate_model(data, target, y_intermediate, 'SVC', list_grouped_positions)

In [ ]:
plot_confusion_matrix(actual_targets, predicted_targets, 'SVC', list_positions)

In [ ]:
actual_targets, predicted_targets = evaluate_model(data, target, y_intermediate, 'MLP', list_grouped_positions)

In [ ]:
plot_confusion_matrix(actual_targets, predicted_targets, 'MLP', list_positions)